# Job application for senior bioinformatician at Turbine

## Bulk RNA-Seq profiling of JQ1 treatment on xxx cell line

The project goal was to quantify the mRNA expression changes of JQ1 treatment in triple-negative breast cancer cell line (). This analysis has 4 parts, which will be descrbed in more details:
  * i) summary notebook (this one)
  * ii) data preprocessing notebook (bin/RNA_seq_data_preprocessing.ipynb)
  * iii)RNA quantfication from raw reads (using SALMON and STAR)
  * iv) Statistical analaysis and visualizatin of the results (DEsSeq2). 



# Bulk RNA-Seq data processing
Next generation sequenceing becaome routinly and easyilx acccessbale for research groups, resulted in the large-scale accumulating of sequencing information in databases. Past decades many tools have proposed and it is not trivial to choose the 'best' tool combination or bioinformatic pipeline for a certain application. Frotunatelly many effort have been taken to support and build such pipelines, especially for RNA-Seq, which is one of the most commonly used technology for investigating the transcription profile of a living organisms. Currently these pipeline tools are the state-of-the art approaches, because they not only help to build a robust pipeline, but they also support the scalibility (via abstractions), which crucial in applications, where the goal is to process 1000s or even more samples.
A good example is Snakemake [2] or Galaxy [3], for a review see [4]. Here on the otherhand I choose to go with a custom, POC pipeline for demonstration purposes, since we are dealing with bioinformatics job application. The first part of the pipeline is done via python, second via bash (i.e. genome indexing) and python, third part (statistical evaluation) in R.

## Infrastructure

All the calculation was carried out in cloud enviremnt (microsoft azure and google cloud), where both tradition (IaaS) and serverless technologies (like cloud-run) were used. The input data is deposited in the cloud (link). The software infrastrucutre (neccessery packages, tools, etc.) are provided via docker. All the codes, notebooks, docker files are deposited in a github repository.
The containers also available publicly and the exact versions of the softwares and librarires can be extracted. I belive this is important otherwise the reproduibility is much harder to be guaranteed. 
Note however, these submited codes, containers are for demonstration purposes only, not suitable for prduction usage (for example the containers can and should be optimaized, security also wasn't addressed). There are many limiations in this version, which can be elimated, at least partly, using propre, CWL like pipeline managers (i.e. collecting input paramters). Ideally such project not only use version control managers like github, but also proper CI/CD pipelines and integrations (like Jenkins).

## Methods (summary)
The general goal is to understand how the JQ1 treamnt change the transcirptioimc landscape of the cell. There are many ways to carry out an RNA-Seq analysis. First, one could choose from de-novo or 'reference' based methods. Personally I was really interested wheter using the recently pubshled telomer-to-telomer reference genome (CHM13v2) [5] gives new insgight about the underlying mechamins of JQ1 treatments. This new reference contains more then 2000 new genes (99 protein coding genes as well), which likely to be relevant in cancer-related processes. However, in subseqent statistical analysis in DeSeq2 failed due to the proper support of annoation packages in R. The solution is compliing the neccessery mappings (i.e. from transcripts to genes, genes to GO categories, etc.). 

In this processing pipeline I used reference based tools, for example STAR [6] and SALMON [7]. The GRCh38.p13 reference genome was used withs it's ensemble provided gene annotation. Note that the gene annotations have significant effect on the final outputs (i.e. significantly differential expressed genes). STAR  is a trandional read mapping tool tailerd to rna-seq like data (i.e. by providing splice aware alignments to reference genome), while SALMON is a k-mer (transcriptome) indexing based method, with increasing popularity. 

Also there is a wide variaty of possible framework of dealing with raw RNA-Seq data like edgeR [8], DESeq2 [9], etc. In this application I used DESeq2. This framework provides many function for visualizing and analizing the rna-seq2 data. It provides a flexible statistical framework for modelling and comparing count data. Here we used a very-sample case-control description of the experiment, note however it could handle much more complex scenarine like one is presented in the original paper [10]. This experiment contains technical replicates of the treatments, which is recommened to be collapsed. Demonstration resans, I skipped that step (better visualization and enrichment analysis).






# Background





# Results



# References
